In [11]:
from openpyxl import load_workbook
import pandas as pd
import numpy as np
from calendar import monthrange, datetime
import re

In [12]:
def getIndexes(dfObj, value):
     
    # Empty list
    listOfPos = []
     
    # isin() method will return a dataframe with
    # boolean values, True at the positions   
    # where element exists
    result = dfObj.isin([value])
     
    # any() method will return
    # a boolean series
    seriesObj = result.any()
 
    # Get list of column names where
    # element exists
    columnNames = list(seriesObj[seriesObj == True].index)
    
    # Iterate over the list of columns and
    # extract the row index where element exists
    for col in columnNames:
        rows = list(result[col][result[col] == True].index)
 
        for row in rows:
            listOfPos.append((col, row))
             
    # This list contains a list tuples with
    # the index of element in the dataframe
    return listOfPos

In [13]:
#parse conditional significations
wb = load_workbook(filename="input/report_2022-11-16_formatted.xlsx")
ws = wb.active
sheet = pd.DataFrame(ws.values)

#find indexes of headers of symbols and descriptions in table
symbol_headers = getIndexes(sheet, 'Символ')
description_headers = getIndexes(sheet, 'Описание')

#indexes of end of symbols and descriptions in table 
symbols_end, descriptions_end = [], []

#result dictionary of symbols and descriptions
condition_significations = {}

#for every column of symbols and descriptions
for symbol_header, description_header in zip(symbol_headers, description_headers):
    #find end of table (None symbol after symbols / descriptions)
    symbols_end = [symbol_header[0], symbol_header[1] + np.where(pd.Index(sheet[symbol_header[0]][symbol_header[1]:]).get_loc(None) == True)[0][0]]
    descriptions_end = [description_header[0], description_header[1] + np.where(pd.Index(sheet[description_header[0]][description_header[1]:]).get_loc(None) == True)[0][0]]

    #get all symbols and descriptions of current columns
    symbols = sheet[symbol_header[0]][symbol_header[1]+1:symbols_end[1]]
    descriptions = sheet[description_header[0]][description_header[1]+1:descriptions_end[1]]

    #add symbols / descriptions in resulting dictionary
    for symbol, description in zip(symbols, descriptions) :
        try:
            condition_significations[sheet[symbol_header[0]][symbol_header[1]]].append(symbol)
            condition_significations[sheet[description_header[0]][description_header[1]]].append(description)
        except KeyError:
            condition_significations[sheet[symbol_header[0]][symbol_header[1]]] = [symbol]
            condition_significations[sheet[description_header[0]][description_header[1]]] = [description]

condition_significations = pd.DataFrame(condition_significations)
condition_significations.insert(0, 'id', condition_significations.index)
# condition_significations.set_index('id', inplace=True)
condition_significations.to_csv("output/Условные обозначения.csv", sep=',', index=False)
condition_significations

,id,Символ,Описание
0,0,"""",высший и низший уровень за месяц в один день
1,1,#,изменение ледовых условий техническими средствами
2,2,&,ледостав с торосами
3,3,(,закраины
4,4,),забереги
5,5,*,редкий шугоход
6,6,/,искажение уровня воды естественными или искусс...
7,7,:,сало
8,8,;,внутриводный лед
9,9,@,плавучий лед


In [14]:
def all_dates_in_year(year=2019):
    for month in range(1, 13): # Month is always 1..12
        for day in range(1, monthrange(year, month)[1] + 1):
            yield datetime.date(year, month, day)

In [15]:
#parse water level

wb = load_workbook(filename="input/report_2022-11-16_2020_formatted.xlsx")
ws = wb.active
sheet = pd.DataFrame(ws.values)

year_header = getIndexes(sheet, 'Год')[0]
year = sheet[year_header[0] + 1][year_header[1]]

day_header = getIndexes(sheet, 'Число')[0]
days_start = [day_header[0], day_header[1] + np.where(pd.Index(sheet[day_header[0]][day_header[1]:]).get_loc(1) == True)[0][0]]
days_end = getIndexes(sheet, 'Декада')[0]

month_header = getIndexes(sheet, 'Месяц')[0]
monthes_start = [month_header[0], month_header[1] + 1]
monthes_end = [monthes_start[0] + 12, monthes_start[1]]

water_level = {'Дата': [], 'Уровень воды': []}
water_characteristics = {'Дата': [], 'id характеристика воды': []}

for i in range(monthes_end[0] - monthes_start[0]):
    month = sheet[monthes_start[0] + i][monthes_start[1]]
    for j in range(days_end[1] - days_start[1]):
        day = sheet[days_start[0]][days_start[1] + j]
        try:
            date = datetime.date(year, month, day)
            value = sheet[monthes_start[0] + i][days_start[1] + j]
            
            if isinstance(value, str):
                tokens = value.split(' ')
                value = int(tokens[0])
                characteristic = tokens[1]
                #баг при "прмз" и "прсх" и любых других, состоящих из >1 символа 
                for symbol in characteristic:
                    if (len(condition_significations.loc[condition_significations['Символ'] == symbol]['id'].values) != 0):
                        water_characteristics['Дата'].append(date)
                        water_characteristics['id характеристика воды'].append(condition_significations.loc[condition_significations['Символ'] == symbol]['id'].values[0])


            water_level['Дата'].append(date)
            water_level['Уровень воды'].append(value)

        except ValueError:
            break

water_level = pd.DataFrame(water_level)
water_characteristics = pd.DataFrame(water_characteristics)

water_level.to_csv("output/Уровень воды.csv", index=False)
water_level

,Дата,Уровень воды
0,2020-01-01,88
1,2020-01-02,96
2,2020-01-03,108
3,2020-01-04,115
4,2020-01-05,104
...,...,...
361,2020-12-27,57
362,2020-12-28,45
363,2020-12-29,58
364,2020-12-30,49


In [16]:
water_characteristics.to_csv("output/Характеристика воды.csv", index=False)
water_characteristics

,Дата,id характеристика воды
0,2020-01-01,13
1,2020-01-14,12
2,2020-01-16,12
3,2020-02-01,13
4,2020-02-24,12
5,2020-03-02,13
6,2020-03-31,12
7,2020-04-01,13
8,2020-04-17,12
9,2020-05-08,13


In [17]:
#parse fields of rainfall and temperatures

with open('input/wr149976/fld149976a0.txt') as f:
    lines = f.readlines()

fields = []
for line in lines:
    buf = ' '.join(line.split()).split()
    buf[3] = ' '.join(buf[3:])
    buf[4:] = []
    fields.append(buf)

field_names = []
for row in fields:
    field_names.append(row[3])

field_names

['Индекс ВМО',
 'Год',
 'Месяц',
 'День',
 'Общий признак качества температур',
 'Минимальная температура воздуха',
 'Средняя температура воздуха',
 'Максимальная температура воздуха',
 'Количество осадков']

In [18]:
#parse rainfall and temperature

with open('input/wr149976/wr149976.txt') as f:
    lines = f.readlines()

rainfall = {}

for line in lines:
    buf = ' '.join(line.split()).split()
    for key, value in zip(field_names, buf):
        try:
            rainfall[key].append(value)
        except KeyError:
            rainfall[key] = [value]

rainfall = pd.DataFrame(rainfall)
rainfall[['Индекс ВМО', 'Год', 'Месяц', 'День']] = rainfall[['Индекс ВМО', 'Год', 'Месяц', 'День']].astype(int)
rainfall[['Общий признак качества температур'
        , 'Минимальная температура воздуха'
        , 'Средняя температура воздуха'
        , 'Максимальная температура воздуха'
        , 'Количество осадков']]                 = rainfall[['Общий признак качества температур'
                                                           , 'Минимальная температура воздуха'
                                                           , 'Средняя температура воздуха'
                                                           , 'Максимальная температура воздуха'
                                                           , 'Количество осадков']].astype(float)

date = []
for year, month, day in zip(rainfall['Год'], rainfall['Месяц'], rainfall['День']):
    date.append(datetime.date(year, month, day))

rainfall['Дата'] = date
rainfall = rainfall[['Индекс ВМО', 'Дата', 'Общий признак качества температур', 'Минимальная температура воздуха', 'Средняя температура воздуха', 'Максимальная температура воздуха', 'Количество осадков']]
rainfall
temperatures = rainfall[['Индекс ВМО', 'Дата', 'Общий признак качества температур', 'Минимальная температура воздуха', 'Средняя температура воздуха', 'Максимальная температура воздуха']]
rainfall = rainfall[['Индекс ВМО', 'Дата', 'Количество осадков']]

rainfall.to_csv("output/Количество осадков.csv", index=False)
rainfall

,Индекс ВМО,Дата,Количество осадков
0,34561,2020-01-01,0.0
1,34561,2020-01-02,0.0
2,34561,2020-01-03,0.0
3,34561,2020-01-04,0.0
4,34561,2020-01-05,0.1
...,...,...,...
361,34561,2020-12-27,8.1
362,34561,2020-12-28,0.0
363,34561,2020-12-29,0.4
364,34561,2020-12-30,0.4


In [19]:
temperatures.to_csv("output/Температура.csv", index=False)
temperatures

,Индекс ВМО,Дата,Общий признак качества температур,Минимальная температура воздуха,Средняя температура воздуха,Максимальная температура воздуха
0,34561,2020-01-01,0.0,-0.3,0.9,2.6
1,34561,2020-01-02,0.0,-3.2,-0.4,1.4
2,34561,2020-01-03,0.0,-3.9,-1.6,0.4
3,34561,2020-01-04,0.0,-0.9,0.5,3.5
4,34561,2020-01-05,0.0,-1.7,0.0,2.1
...,...,...,...,...,...,...
361,34561,2020-12-27,0.0,-1.2,1.0,2.8
362,34561,2020-12-28,0.0,-4.6,-2.9,-0.4
363,34561,2020-12-29,0.0,-9.8,-7.2,-2.5
364,34561,2020-12-30,0.0,-8.9,-6.7,-5.3


In [20]:
with open('input/wr149976/statlist149976.txt') as f:
    lines = f.readlines()
    
stations = {'Индекс ВМО' : [], 'Город' : [], 'Страна' : []}
# stations = pd.DataFrame(columns=['Индекс ВМО', 'Город', 'Страна'])
for line in lines:
    buf = ' '.join(line.split()).split()
    stations['Индекс ВМО'].append(int(buf[0]))
    stations['Город'].append(buf[1])
    stations['Страна'].append(buf[2])

stations = pd.DataFrame(stations)
stations.to_csv("output/Метеостанции.csv", index=False)
stations

,Индекс ВМО,Город,Страна
0,34561,Волгоград,Россия
